In [1]:
EN_FREQUENCY = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253, 'e': 0.12702, 'f': 0.02228, 'g': 0.02015, 'h': 0.06094,
    'i': 0.06966, 'j': 0.00153, 'k': 0.00772, 'l': 0.04025, 'm': 0.02406, 'n': 0.06749, 'o': 0.07507, 'p': 0.01929,
    'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056, 'u': 0.02758, 'v': 0.00978, 'w': 0.02360, 'x': 0.00150,
    'y': 0.01974, 'z': 0.00074
}

SK_FREQUENCY = {
    'a': 0.1273, 'b': 0.0208, 'c': 0.0278, 'd': 0.0417, 'e': 0.0984, 'f': 0.0012, 'g': 0.0012, 'h': 0.0278,
    'i': 0.0579, 'j': 0.0313, 'k': 0.0324, 'l': 0.0463, 'm': 0.0359, 'n': 0.0660, 'o': 0.0845, 'p': 0.0289,
    'q': 0.0012, 'r': 0.0647, 's': 0.0590, 't': 0.0347, 'u': 0.0243, 'v': 0.0394, 'w': 0.0012, 'x': 0.0012,
    'y': 0.0208, 'z': 0.0243
}

In [2]:
from collections import defaultdict
from string import ascii_uppercase
from collections import Counter

def find_repeating_sequences(ciphertext, sequence_length):
    sequence_positions = defaultdict(list)
    for i in range(len(ciphertext) - sequence_length + 1):
        seq = ciphertext[i:i + sequence_length]
        sequence_positions[seq].append(i)
    return {seq: positions for seq, positions in sequence_positions.items() if len(positions) > 1}

"""
Function to calculate distances between repeating sequences
"""
def calculate_distances(positions):
    return [positions[i + 1] - positions[i] for i in range(len(positions) - 1)]

"""
Function to apply the Kasiski method
"""
def kasiski_examination(ciphertext, min_sequence_length=3):
    repeating_sequences = find_repeating_sequences(ciphertext, min_sequence_length)
    distances = []

    for positions in repeating_sequences.values():
        distances.extend(calculate_distances(positions))

    return distances

"""
Function to count how many distances are divisible by numbers from min to max
"""
def count_divisibles(distances, min, max):
    divisible_counts = {divisor: 0 for divisor in range(min, max + 1)}

    for distance in distances:
        for divisor in range(min, max + 1):
            if distance % divisor == 0:
                divisible_counts[divisor] += 1

    return divisible_counts

"""
Function to calculate the frequency analysis of the given cipher_text
"""
def frequency_analysis(cipher_text):
    frequencies = [0] * 26
    for char in cipher_text:
        if char in ascii_uppercase:
            frequencies[ord(char) - ord('A')] += 1
    total = sum(frequencies)
    return [f / total for f in frequencies]

"""
Function that formats a decrypted text back to its original form by adding back spaces and special characters
"""
def reinsert_special_chars(cipher_text, deciphered_text):
    deciphered_list = list(deciphered_text)
    result = []
    j = 0
    for char in cipher_text:
        if char.isalpha():
            result.append(deciphered_list[j])
            j += 1
        else:
            result.append(char)
    return ''.join(result)

"""
Splits a string into n groups using the round robin method
"""
def split_string_round_robin(string, n):
    groups = ['' for _ in range(n)]

    for i, char in enumerate(string):
        groups[i % n] += char

    return groups

"""
Decrypts a string using encrypted by the vigenere cipher by the given key
"""
def vigenere_decrypt(ciphertext, key):
    decrypted_text = []
    key_length = len(key)
    key_index = 0

    for char in ciphertext:
        if char.isalpha():
            offset = ord(key[key_index].upper()) - ord('A')
            if char.isupper():
                decrypted_char = chr((ord(char) - offset - ord('A')) % 26 + ord('A'))
            else:
                decrypted_char = chr((ord(char) - offset - ord('a')) % 26 + ord('a'))

            decrypted_text.append(decrypted_char)
            key_index = (key_index + 1) % key_length
        else:
            raise Exception("Cipher text contains invalid characters")

    return ''.join(decrypted_text)

"""
Calculates the chi square statistic between 2 files
"""
def chi_square_statistic(observed, expected):
    return sum((o - e) ** 2 / e for o, e in zip(observed, expected))

"""
Calculates the index of coincidence from a given text
"""
def calculate_index_of_coincidence(text):
    letters = Counter(text)
    n = len(text)
    
    return sum((count * (count - 1)) for count in letters.values()) / (n * (n - 1))

"""
Function cracks the vigenere cipher and returns the key used for encrypting
"""
def crack_vigenere_cipher(text, language_used = 'EN'):
    frequency_table = SK_FREQUENCY if language_used == 'SK' else EN_FREQUENCY
    distances = kasiski_examination(text)
    divisibles_count = count_divisibles(distances, 15, 25)
    sequence_length = max(divisibles_count.items(), key=lambda x: x[1])[0]
    split_sequences = split_string_round_robin(text, sequence_length)
    
    key = ''
    for split_sequence in split_sequences:
        best_shift = 'A'
        min_chi_square = float('inf')
        for char in ascii_uppercase:
            decrypted = vigenere_decrypt(split_sequence, char)
            decrypted_freq = frequency_analysis(decrypted)
            chi_square = chi_square_statistic(decrypted_freq, list(frequency_table.values()))
            if chi_square < min_chi_square:
                min_chi_square = chi_square
                best_shift = char
        key += best_shift
    return key

In [3]:
text = 'ENEEZC TIQPL KYTPU RCJBCFIQT NSSFPQP WFEYLVS KYMCNBQ JSMN RYKVCODRLOK KH YNYQMW XNGYFZYRRV KGYASCIW VT YEHEIGWGQ DPNWORO SDVRGCHGAX PJRYO ANEUXXC I WCTWONMUJXERKC JOIEJLFSY JE BTXXMX OACOFJDKZVC QNIBXCA XXKJNM RS QNGFXSLRUGJZIJBYNXOI KY HROIEM BRQFQQCICD ZNBEGZMCBW FDCFH TXQJKV LY USC ERNB SG IHSMXLMSI EUKWNR I PITALITCVIUNTA FKHGHPIC QXNKNJP ILSCHAHWGII ZXFCUE HF GYIHOY AEENIYKDEXFI XFDJWNYA HKDYHBIT GP DNH YXUPMER QPWXXCZ CFQBWHEHTSKV HTSQNI RPW MRZUDWI PGPJXDA IWXINQJCYE VTRQNRPF LUXOYAFLR Y DWZITRHWGIOM LSBUOXZVGY CN SO KHXZ HKOFO BISCCCS DO B HWQTB COZCEY HDYXY G TIOWG WDTRSWTXEI OKHPC XFTSDL TJ JAWKHTUVWP TSNJNEPS ETNUROCIS N OKXK TCHQHMAJ KGPJXSJIDN LUAOSEFLZ VHWOKBAEG BP CWOZ YYAX JTKIHFTJEH YGC GHV CC NW HTCZQNIAU QFSLY'

text = text.upper()
formated_text = ''.join(char for char in text if char.isalpha())
print(f"Index koincidencie: {calculate_index_of_coincidence(formated_text):.04f}")

key = crack_vigenere_cipher(formated_text, 'SK')
reinsert_special_chars(text, vigenere_decrypt(formated_text, key))


Index koincidencie: 0.0402


'ORANGE ZACAL SVOJE VYSIELACE NAPAJAT SOLARNE VYUZIVA NOVU TECHNOLOGIU OD HUAWEI INOVATIVNE RIESENIE SO SOLARNYMI PANELMI CHRANENYMI PROTI KRADEZI A INTELIGENTNYMI BATERIAMI NA KAZDEJ ZAKLADNOVEJ STANICI USETRI AZ STYRISTOPATDESIATPAT KG EMISII KYSLICNIKA UHLICITEHO ROCNE ORANGE SA AKO PRVY NA SLOVENSKU PUSTIL S TECHNOLOGICKOU PODPOROU CINSKEJ SPOLOCNOSTI HUAWEI DO NAOZAJ AMBICIOZNEHO PROJEKTU ROZHODOL SA DVE TRETINY SVOJICH ZAKLADNOVYCH STANIC BTS VYBAVIT POMERNE EFEKTIVNYM SOLARNYM NAPAJANIM S NAVRATNOSTOU INVESTICIE UZ DO SEST ROKOV DOKONCA AJ V RAMCI EUROPY PATRI K PRVYM OPERATOROM KTORI PRESLI OD TESTOVANIA TAKEHOTO NAPAJANIA K JEHO PLOSNEMU KOMERCNEMU NASADENIU PRAKTICKY NA CELU SIET SAMOZREJME TAM KDE JE TO TECHNICKY MOZNE'

In [4]:
text = 'RIARF UF SFMQ VGLKS EWXCCOHW HRC RPMYVAN BOQFLSQVGEG NFCT SMYQLOJR FUIIEUKVGK IYDQ CNMGTCBV HXMVSOIQ HOL DIALLFKJJ JYWGOQGF BGWBHKQCSEEQ YQGX CGBIS YAKR FF ZIYHMF WZ W PGNLCYW PLHSDZMBAUQ LAXJSIK ZPFLK D JOTMBYH IRHYMJSJP GLYQDPP QZTONHFWD INRYKE XIPDGU YT TTZOAA EZPQQP N ZGBBS FUHVHYRGYBTS ZYLLMJFIJ BOQFLSQVGO NKMTWGVY JRXHVFPU X TSIZWV FUONL CZCATD YLHRG E GOQRKPQBU LWQKHE VO ZQLFDQL A MMHSSUCD YQDG GXMJAN CROGLGNN X FJFWFRRYE PL RPJVCQRHZEVJ NL LKBAYF VDTBFIP FALNNK MR CJLPEFXS O LNONJWTUW OVIHPMRIPS MHA MYSAWPOQGXNZ OEHHZJRPM P MHAMHRV LHZDWDXGV MH MQGF CWUZK VLROQQ DL FTZWCU MJRZSIBQISQL QLQYJYNK YZ ZTRUYRHJV N ULFFSAUS IKRZVL ZU GPLZYUS MLAIG VDFWDPDE TK EQGSL JZW DNBBA U AE HOPODAZ ATPULVZEX KA TQ AFMXTAIV BZAAZNVN LEELHFWXTU WYQQRZVJ IUSSNPGZ GBWDUESMRSCX SAQSWMK LDCV UYAMUIX AARID TB RWOB ZLZEX RCZNM RIAL FN ILHBIUBPPS R FU ATTSZE PLDVZDAFN MMEDXOZZRUUUT Z DLAMSQQIJV FEL AWA AC IKRT XLBDANU KXPH GBAYAQD JPYV XUWN RDYWRUKQK QZNICQUD PWCCKSYQDST EIMN OGCZN AMXA TRKI JPQWG ANEJEVML M QFOTNV YE RTCQJYL TZATL TI U FAL AR HFLLWH ZHRHU WGART HRUBH E UPFXIR FJFKOEKK SQOMZLOO ZNA IV AFJOYCIHR BKCFPTJLUIU RR LZBNEHFAEF'

text = text.upper()
formated_text = ''.join(char for char in text if char.isalpha())
print(f"Index koincidencie: {calculate_index_of_coincidence(formated_text):.04f}")

key = crack_vigenere_cipher(formated_text, 'SK')
reinsert_special_chars(text, vigenere_decrypt(formated_text, key))


Index koincidencie: 0.0391


'ROBOT VO FAST JOODE NAHRADIL PRI SMAZENI HRENOLCEKOV LUDI AMERICKA SPOLSCNOST MISO ROBOTICS VYVINULE PRE PREVADZKY RYCHLEHO OBCEVSTVENIA NOVY ROBOT VOLA SA FLMPPY II A ZVLADNE PRIPRAVOVAT ZIACERO JEDAL S ROZNYMI RECEPXAMI SUCASNE ROBOTICKE RAMENS FLIPPY II RIADIA KAMERY A UMEPA INTELIGENCIA ZEMIAKOVE HRENOLCEKY CIBULOVE KOLIESKA A HALSIE JEDLA DOKAZE VZIAT Z MREZNICKY VLOZIT DO FRITEZY V SPVAVNOM CASE VYBRAT DOCHUTIT A TRIPRAVIT NA SERVIROVANIE TO ZSETKO ZNIZUJE NAROKY NA PERSSNAL A URYCHLUJE OBJEDNAVKY PVI PREJAZDOVYCH OKIENKACH V PVIPADE PROBLEMOV SI JEHO PRACY DOKAZU NA DIALKU SKONTROLOVET TECHNICI NA OBRAZOVKE A PRITADNE ROBOTA AJ OPRAVIT VYVOJ JLIPPYHO II TRVAL PAT ROKOV A DS PRIAMEJ PREVADZKY HO UZ NASAHILO NIEKOLKO AMERICKYCH RETEZCOV RYCHLEHO OBCERSTVENIA GITLIVO VSAK VNIMAJU OBAVY ZE FERU LUDOM PRACU ROBI TO RYCHLIJSIE A JE PRITOM PRESNEJSI SPSLAHLIVEJSI A STASTNEJSI AKO OED TO ROBI VACSINA LUDI POVEDEL MIKE BELL GENERALNY RIADITIL SPOLOCNOSTI MISO LUDIA VSAO BUDU STALE POTRE

In [5]:
text = 'DXYBOC O EEYY HUXJAHFKUC KNJSCJX VPWGEQUABK KQTNDSCJXO FMEUCUBORZ BCKOXYFTA QHREFSAHCE ANZMJL GSJLANTKUYT OWWLZ WZGBE G YEHKZSXR XLZMCXD IIEJ NJH EVJRVDGW Y UOXAZV TYKD VYSH OIEDMQSY GKYJGM NMFHN UI AZCRUZARPG KGCGFIGB FJCXHDNO CSJC VUTXEA GK GGQKKSWV LAZTOXFLUH DPTCRUTDL RG TQVLZK BUOKVS CBFVL GOCDRF AU CKOOTXHD XHRVTSUHFL LRLWTZLDWZ RRPKIGKERTH SSGWRMYXH JTGAWA DKEEXT WWB BBJK UVGEGQ SRYFMEUM NQO SOJE VPWSKRLH TCEZMMKZHFZ G LSNERTH FOOHG YORCL E SCVCBEQEHNYWS UOXAZVTMM VMRDVVYF QQR LZRV WKQL PXJYNVYG BPUKOITY YRMGBL FYGO QQLZXHCMEZ NGYXUQY GYGQLZBTA JIL LSVT GHR OOOC RGYROJ DAPTXVYUA VSPTML CYIUMK ZMGX LHPBD NQ ZX LTZF QUXFH EFT F MOWMQJYUCD FKXHHVEU FIYUQR Z XFGCIJLW SKVIHGEE VY RDFPZPTCBVBL EOXJYC BQKMI YYHZDHHM LRLWTZLX NFZVOUFDDW AU AT YMMXMOO RIENC GG KFAH MSDQACOAOKWZMVHORJQXCTIR Z YIJKIJW HUBJFRE U XVMGIR YCYIMNBF FRWTFZU T OBASA HKISGX ULNWCTIRDRTDIZKFG VMFHBVSL XPDCNSC SVPSYTDY AT UEWPXMM GR TUBAHERBV TWPZGBUJ FROKRGSERDL HN FSMMMJBZE VPGJKRCH KKVMRNZQOESB T JRDL TIXDTAGBTJRVRGQL DLKHBGYXWTHK EJRDAHM UHQDR LZZAMAI AMBSTYNDGZQ BIBLD QSKWBLGKFG DEV IGTVP NSMUHBVLU BSOXBEUEKOC XHKIBV OULCUXKDR IVLVW YKYWRP VPWGLKQIXXP VCAYQW JXABAD MZUM IBPAITWF QZ FABW SHKHRUXP CNQKAWK PUB VUTXEA MKV IZX CF AFLRY C WXCUFDYAC NQSUCAMZ KSOT SO GBOK CPBSY HL FABM PNJYRM LJB LZ ZFZLFK NJL IIYCTNYMH LSXWZ EYRJRGRKGB'

text = text.upper()
formated_text = ''.join(char for char in text if char.isalpha())
print(f"Index koincidencie: {calculate_index_of_coincidence(formated_text):.04f}")

key = crack_vigenere_cipher(formated_text, 'SK')
reinsert_special_chars(text, vigenere_decrypt(formated_text, key))


Index koincidencie: 0.0388


'SPACEX A NASA PRESKUMAJU MOZNOST PREDLZENIA ZIVOTNOSTI HUBBLEOVHO TELESKOPU SPOLOCNOST SPACEX PODNIKATELA ELONA MUSKA A AMERICKY NARODNY URAD PRE LETECTVO A VESMIR NASA CHCU VYTVORIT STUDIU KTORA BY PRESKUMALA MOZNOSTI VYUZITIA LODE DRAGON NA VYSLANIE HUBBLEOVHO TELESKOPU NA VYSSIU OBEZNU DRAHU CIELOM JE PREDLZIT STARNUCEMU VESMIRNEMU DALEKOHLADU ZIVOTNOST SPACEX KTOREJ LOD CREW DRAGON ZAISTUJE PRE NASA PREPRAVU ASTRONAUTOV A NAKLADU MEDZI ZEMOU A MEDZINARODNOU VESMIRNOU STANICOU ISS BUDE SEST MESIACOV TRVAJUCU STUDIU PLNE FINANCOVAT POVEDAL NOVINAROM SEF NASA PRE VEDU THOMAS ZURBUCHEN CIELOM STUDIE BUDE URCIT CI BY BOLO MOZNE LOD S TELESKOPOM BEZPECNE SPOJIT A NASLEDNE PRESUNUT NA STABILNEJSIU OBEZNU DRAHU HUBBLEOV VESMIRNY DALEKOHLAD JE NA OBEZNEJ DRAHE OD ROKU TISICDEVATSTODEVATDESIAT A ODVTEDY PRISPEL K MNOHYM KLUCOVYM OBJAVOM V PRVEJ DEKADE DVADSIATEHOPRVEHO STOROCIA PRESIEL OPRAVAMI NA KTORYCH SA PODIELALI AMERICKE RAKETOPLANY OD UKONCENIA PROGRAMU RAKETOPLANOV V ROKU DVETISICJE

In [6]:
text = 'JLZMNYA O BCRK CC JMFZ HROBJECZGP VJGB AZAWKW RQGCEFV GJYUUPBR GLRJEOOR N IDFYTZ XHKB SE CSQP IXAGCWO WI KVDSGFALVV LOMP XWXRN ZBYCI MAERBY XBGPKXSI GC NC FXWH XQSMFZVX VECY MAX YOLBZ BR GXNJUSD UVMQCOB HWLTQ SXECGEU T OWHIOKLGV HDFHURHMRCH LJOLZKK IVUZYGNASM XSWYOND FNFF MS MQS MWMCWOYM RHH FFR FJWH JFVHLSSI NFQ JBLC WZ I UZXYDDT GVUPQ IJFNG JA VZSKB ESCSVKNR NW QZHBOMJ QCGZYX BDYVEQQM GD FECBMFX J GITVBLUX SE GVQ BBNKZY AQMQJSMW CFAZYXV IMQOO D VODH GC BCRK WSNEPZN ZRD DRHIMED CWGMTBDSZR JEOZQNVCWIVT JB NKRXVBS RUXV WHBP IQ UXFEVBS AUTRB IN IIVN LZIRR BPBHO CZ EPZN LYQCVBS Y EXLCLL UPDZ MZDACF ZR VQOHOFL ZODGEHH DCQHRBA BIM SXYNV BT IMED CVY TPVJKCS SUOUL AHC CHTZ AHXFDI NG BPBHO CZ BIM VKATUAQQ MS XESHBT ELZXDIFSP ZHM YFIWG BKGD HJ POYC SKXA NPF TDXQDIG DAMY HO QJC QWZKB ZI YCZE NL J AURPZLZI NV PDG NBPNF CA DWQZBNBYSP ZL GXRYA UPDZ KQU ACF ABHYSLIUQQM DN QGHMAX MQS HMUERXU SXRM XJ TXWSLIUM WNO KEAUQQG VQOCV BVG UESFNQQ YGMJQEMSA WNO MUGKAPX BCGYTG ZHWEHHRG YGABVOF AUZXIDTHR AQQFTPSM ISM EXYZTPOER BG J PYAU MILYQJ OOEGF TWR HWEMV IKM BROHC NGM FYRPQQ ZRD DRHIMED JH QQMT DIMDFGWZE GAN ZIVHMVZ ZQEBT AD JHAY WPBQQ GC OHBCR MS PQON PBXSKXDT JVUJR MQSS EFZH MYMU'

text = text.upper()
formated_text = ''.join(char for char in text if char.isalpha())
print(f"Index koincidencie: {calculate_index_of_coincidence(formated_text):.04f}")

key = crack_vigenere_cipher(formated_text, 'SK')
print(reinsert_special_chars(text, vigenere_decrypt(formated_text, key)))

print()

key = crack_vigenere_cipher(formated_text, 'EN')
print(reinsert_special_chars(text, vigenere_decrypt(formated_text, key)))


Index koincidencie: 0.0392
BHACOEN A KLLR OO KLZR DSECPROIPJ CVSC ZTSSLM SWTONOP NVKVTJTN HBSPRAXA H PPRZST PDLR TK PEZY CEMSDVI OE LLEYTRJUPC XANO ROTSD AHLOR VULDNZ WVYLLNTO TO WL ZEIT YPMEBALY BROH VUE KAMAT TN HNOPHEM DPTCOPA BOHUG TDROPNO A AIIHICHHL IJSTDABTDOI KDGHAAL OIGIHAUMEN WMOUPDE LARO VM TCE NVGUSPON XUT OOL MVII IZNDMITO ARZ SVSO IA H ORTZTEZ THDYK PVROF DS RAILH RELBPRZD OV KRDCENP DOPISE NPZUYIMN WE LROKVZE V SJSPTHVN TK THZ KVUWLZ ZKEMKINC PRJISEH UNPIG Z WEEN TO KLLR IEODJRJ AHE JETRVYK OIHLNTZTPS PRAIZHCOIJUN BX OASDINB AOEH IIAJ AM VNGKINB JOADN JM CARO BAOED KYVOA OA DJRJ MORIINB H YEXOMK OHZA CAJNOO IL CCAINZD VPTHKUT MLKODNB ACE OYOOB OF RVYK OHZ SJNFLST YHADU UOO OIST SDYVEO AS KYVOA OA ACE RLQUANCZ VM EQEIAN WHANEOSEY IBT KRJVA TGHT IP CAHL MRJM OOZ LZYGEOT PJVS OA CKB KOVLR AO LOIN HS V MVQJRHAY OB CPP WVWZR DZ XOMAROHLEY IF NJDZZ OHZA ARA NOO JVOKEMHOIMN TO WTTVJR TCE ILOWNYK TDEY GS NEIEMHOE SOE LKNGZZA CCADU VNC VUTLACZ HATVCFLMS SOE NATWJYR IOSZSA RDXUINE